In [1]:
import os
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
processed_dir = os.path.join(project_root, "notebooks", "data")

X_train = pd.read_csv(os.path.join(processed_dir, "X_train_scaled_merged.csv"))
y_train = pd.read_csv(os.path.join(processed_dir, "y_train_merged.csv"))      

X_valid = pd.read_csv(os.path.join(processed_dir, "X_valid_scaled_merged.csv"))
y_valid = pd.read_csv(os.path.join(processed_dir, "y_valid_merged.csv"))

X_test = pd.read_csv(os.path.join(processed_dir, "X_test_scaled_merged.csv"))
y_test = pd.read_csv(os.path.join(processed_dir, "y_test_merged.csv"))

In [3]:
mlflow.set_experiment("Customer_Churn_New_Split")


with mlflow.start_run(run_name="Baseline_Logistic_Regression") as run:
    print("\n--- Starting Run: Baseline Logistic Regression ---")

    lgr_params = {
        'class_weight': 'balanced',
        'random_state': 42
    }
    lgr = LogisticRegression(**lgr_params)
    
    mlflow.log_params(lgr_params)

    # --- b. Train the Model ---
    print("Training Logistic Regression...")
    # Make sure y_train, y_valid, y_test are 1D arrays using .values.ravel()
    lgr.fit(X_train, y_train.values.ravel())
    
    # --- c. Evaluate on Validation Set ---
    print("Evaluating on Validation Set...")
    y_pred_val = lgr.predict(X_valid)
    val_f1 = f1_score(y_valid, y_pred_val)
    val_accuracy = accuracy_score(y_valid, y_pred_val)
    
    mlflow.log_metric("validation_accuracy", val_accuracy)
    mlflow.log_metric("validation_f1_score", val_f1)
    print(f"Validation F1-Score: {val_f1:.4f}")

    # --- d. Evaluate on Test Set ---
    print("Evaluating on Test Set...")
    y_pred_test = lgr.predict(X_test)
    test_f1 = f1_score(y_test, y_pred_test)
    test_accuracy = accuracy_score(y_test, y_pred_test)
    test_precision = precision_score(y_test, y_pred_test)
    test_recall = recall_score(y_test, y_pred_test)
    
    mlflow.log_metric("test_accuracy", test_accuracy)
    mlflow.log_metric("test_precision", test_precision)
    mlflow.log_metric("test_recall", test_recall)
    mlflow.log_metric("test_f1_score", test_f1)
    print(f"Test F1-Score: {test_f1:.4f}")

    # --- e. Log the Model ---
    mlflow.sklearn.log_model(lgr, "logistic_regression_model")
    
    print("--- Run Finished ---")


--- Starting Run: Baseline Logistic Regression ---
Training Logistic Regression...
Evaluating on Validation Set...
Validation F1-Score: 0.8259
Evaluating on Test Set...


2025/08/24 12:48:39 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Test F1-Score: 0.8268


2025/08/24 12:48:43 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/08/24 12:48:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


--- Run Finished ---


#  Experiment Results: Baseline Logistic Regression (Resampled Data)

## ✅ Key Findings
- **Excellent Performance**: Test F1 = **0.827**, a strong result for a baseline model.  
- **Stable & Generalizable**: Validation F1 = **0.826** vs. Test F1 = **0.827** → no overfitting.  
- **Root Cause Fixed**: Confirms that earlier failures were due to **data drift**, not model choice.  
- **Reliable Baseline**: We now have a trustworthy dataset structure for future experiments.  

---

## 📈 Performance Metrics
| Metric     | Validation | Test   |
|------------|------------|--------|
| F1-Score   | 0.8259     | 0.8268 |
| Accuracy   | 0.8145     | 0.8157 |
| Precision  | -          | 0.8642 |
| Recall     | -          | 0.7925 |

---

## 🚀 Next Steps
- Build on this solid baseline by training **more complex models** (e.g., Random Forest).  
- Goal: Capture richer patterns in the data and surpass the current F1 of ~0.827.  
